In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [2]:
train_data=pd.read_csv('cleaned_data_set/cleaned.csv')

In [3]:
X=train_data.drop(['pedal_cycles','two_wheeled_motor_vehicles','cars_and_taxis','buses_and_coaches','lgvs','all_hgvs','all_motor_vehicles'],axis=1)
y=train_data['lgvs']

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

LSTM

In [7]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

In [8]:
lstm_model.fit(X_train_reshaped, y_train, epochs=30, batch_size=32)

Epoch 1/30
21665/21665 [==============================] - 22s 972us/step - loss: 14859.7979
Epoch 2/30
21665/21665 [==============================] - 21s 965us/step - loss: 14680.6367
Epoch 3/30
21665/21665 [==============================] - 21s 985us/step - loss: 14665.3281
Epoch 4/30
21665/21665 [==============================] - 21s 982us/step - loss: 14658.5693
Epoch 5/30
21665/21665 [==============================] - 21s 988us/step - loss: 14657.9375
Epoch 6/30
21665/21665 [==============================] - 21s 985us/step - loss: 14656.1885
Epoch 7/30
21665/21665 [==============================] - 21s 988us/step - loss: 14654.6641
Epoch 8/30
21665/21665 [==============================] - 21s 987us/step - loss: 14652.4023
Epoch 9/30
21665/21665 [==============================] - 21s 985us/step - loss: 14651.1055
Epoch 10/30
21665/21665 [==============================] - 22s 998us/step - loss: 14649.7744
Epoch 11/30
21665/21665 [==============================] - 21s 989us/step - los

In [9]:
y_pred_lstm = lstm_model.predict(X_test_reshaped)
mse_lstm = mean_squared_error(y_test, y_pred_lstm)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
rmse_lstm= np.sqrt(mse_lstm)
print(f"LSTM Performance: MSE = {mse_lstm:.2f}, MAE Score = {mae_lstm:.2f}, RMSE Score = {rmse_lstm:2f}")


5417/5417 [==============================] - 4s 665us/step
LSTM Performance: MSE = 14756.26, MAE Score = 73.41, RMSE Score = 121.475354


In [10]:
lstm_model.save('models/lgvs_lstm.keras')

GRU

In [13]:
gru_model = Sequential()
gru_model.add(GRU(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mse')

In [14]:
gru_model.fit(X_train_reshaped, y_train, epochs=30, batch_size=32)

Epoch 1/30
21665/21665 [==============================] - 55s 2ms/step - loss: 14801.3623
Epoch 2/30
21665/21665 [==============================] - 63s 3ms/step - loss: 14679.7920
Epoch 3/30
21665/21665 [==============================] - 44s 2ms/step - loss: 14663.9521
Epoch 4/30
21665/21665 [==============================] - 49s 2ms/step - loss: 14657.5518
Epoch 5/30
21665/21665 [==============================] - 63s 3ms/step - loss: 14653.8691
Epoch 6/30
21665/21665 [==============================] - 62s 3ms/step - loss: 14649.9004
Epoch 7/30
21665/21665 [==============================] - 59s 3ms/step - loss: 14646.4805
Epoch 8/30
21665/21665 [==============================] - 29s 1ms/step - loss: 14643.2217
Epoch 9/30
21665/21665 [==============================] - 31s 1ms/step - loss: 14642.1709
Epoch 10/30
21665/21665 [==============================] - 46s 2ms/step - loss: 14639.2178
Epoch 11/30
21665/21665 [==============================] - 59s 3ms/step - loss: 14638.3076
Epoch 12

In [15]:
y_pred_gru = gru_model.predict(X_test_reshaped)
mse_gru = mean_squared_error(y_test, y_pred_gru)
mae_gru = mean_absolute_error(y_test, y_pred_gru)
rmse_gru = np.sqrt(mse_gru)

5417/5417 [==============================] - 6s 1ms/step


In [16]:

print(f"GRU Performance for pedal cycles: MSE = {mse_gru:.2f}, MAE Score = {mae_gru:.2f}, RMSE Score = {rmse_gru:2f}")

GRU Performance for pedal cycles: MSE = 14704.86, MAE Score = 72.80, RMSE Score = 121.263596


In [17]:
gru_model.save('models/lgvs_gru.keras')

In [18]:
y_train = np.array(y_train)
svr = LinearSVR(max_iter=5, random_state=42)
if y_train.ndim == 1:  # Check if it's 1D
    y_train = y_train.reshape(-1, 1)

svm_model = MultiOutputRegressor(svr, n_jobs=-1)
svm_model.fit(X_train, y_train)

MultiOutputRegressor(estimator=LinearSVR(max_iter=5, random_state=42),
                     n_jobs=-1)

In [19]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

y_pred = svm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"SVR: MSE = {mse:.2f}, MAE Score = {mae:.2f}, RMSE= {rmse:.2f}")

SVR: MSE = 16028.91, MAE Score = 92.31, RMSE= 126.61


In [20]:
joblib.dump(svm_model, f'../models/lgvs_svm.pkl')

['../models/lgvs_svm.pkl']